In [2]:
import os
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
df = pd.read_csv('C:/Users/sjcrum/Documents/Natural Language Processing/FinalProject/data/sample_data.csv')
#print(df)

In [4]:
df.groupby(['pat_no'])

In [5]:
df.sort_values(['pat_no', 'claim_no']).groupby(['pat_no'])

In [6]:
df_per_patent= df.sort_values(['pat_no', 'claim_no']).groupby('pat_no', sort=False).claim_txt.apply(' '.join).reset_index(name='new_seq')
print(df_per_patent)
print('\n')
print(df_per_patent.iloc[1,1])
print(len(df_per_patent))

    pat_no                                            new_seq
0  3930271  1. A golf glove comprising at least an index f...
1  3930272  1. In combination with a height adjustable cri...
2  3930273  1. A bed arrangement comprisinga bed frame,a s...


1. In combination with a height adjustable crib or the like of the type having a corner post and an extensible leg slidably connected to the corner post and having a series of holes along its length, a lock comprisingA. a rigid bracket extending around the extensible leg and having sides lying flush against opposite sides of the corner post;B. means for pivotally connecting the bracket sides to the corner post so that the bracket can be swung toward and away from the leg;C. a rigid nose projecting from the bracket and extending toward the leg, said nose being arranged to engage in one of the leg holes when the bracket is swung against the leg so as to lock the leg at a selected position of lengthwise adjustment relative to the corner post, 

In [7]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import TreebankWordTokenizer
import nltk.data
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()

sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
treebank_tokenizer = TreebankWordTokenizer()

doc_list = []

for i in range(len(df_per_patent)):
    doc_list.append(df_per_patent.iloc[i,1])

print(doc_list)


["1. A golf glove comprising at least an index finger receptacle, a middle finger receptacle and a back surface extending from said receptacles in the direction of the wrist, and further comprising another finger receptacle disposed adjacent said index finger receptacle and said middle finger receptacle and overlapping a portion of said back section. 2. A golf glove in accordance with claim 1 wherein said other finger receptacle is attached along its periphery to said index finger receptacle, said middle finger receptacle and said back surface. 3. A glove comprising an index finger receptacle, a middle finger receptacle, a back surface extending in the direction of the wrist, a finger restraining strap extending between said receptacles, and a finger restraining strap attached to said back surface adjacent said index finger receptacle and said middle finger receptacle and aligned in a direction substantially parallel to said finger restraining strap extending between said finger recept

In [10]:
import string
#Read in file, tokenize into sentences, and tokenize into words
exclude = list(set(string.punctuation))
stop = list(set(stopwords.words('english')))


def read_file_and_tokenize(docs):
    tokenized_docs = []
    pattern = re.compile(r'[0-9]+.')
    for doc in docs:
        punkt_sentences = sentence_tokenizer.tokenize(doc)
        filtered = [i for i in punkt_sentences if not pattern.search(i)]
        no_punc = []
        for i in filtered:
            punc_free = ''.join(a for a in i if a not in exclude) 
            no_punc.append(punc_free)
        sentence_words = [treebank_tokenizer.tokenize(sentence) for sentence in no_punc]
        no_stops = []
        for a in sentence_words:
            no_stops_sub = []
            for b in a:
                if b not in stop:
                    stem = porter_stemmer.stem(b)
                    no_stops_sub.append(stem)
            no_stops.append(no_stops_sub)
            
        tokenized_docs.append(no_stops)
    return tokenized_docs

token_docs = read_file_and_tokenize(doc_list)


In [11]:
def term_frequency(corpus):
    terms = []
    for doc in corpus:
        for sentence in doc:
            for word in sentence:
                terms.append(word)
    
    unique_terms = sorted(list(set(terms)))

    doc_counts = []
    
    for doc in corpus:
        flat_doc = [word for sentence in doc for word in sentence]
        counts = []
        for word in unique_terms:
            total = 0
            count = flat_doc.count(word)
            total += count
            counts.append(total)
            
        doc_counts.append(counts)
        
    return doc_counts, unique_terms
    
tf_docs, terms = term_frequency(token_docs)

In [ ]:
# tf_df = pd.DataFrame({'Terms': terms, 'Doc1': tf_docs[0], 'Doc2': tf_docs[1], 'Doc3': tf_docs[2]})
# tf_df.set_index('Terms', inplace=True)
# print(tf_df)    

LDA Model

In [173]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [174]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [175]:
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(0, '0.114*"said" + 0.037*"riser" + 0.037*"side" + 0.032*"pin" + 0.030*"bed" + 0.029*"member" + 0.028*"respective" + 0.026*"pivot" + 0.026*"frame" + 0.020*"rail"'), (1, '0.087*"finger" + 0.078*"receptacle" + 0.066*"said" + 0.031*"index" + 0.023*"comprising" + 0.023*"middle" + 0.023*"back" + 0.023*"golf" + 0.019*"extending" + 0.019*"surface"'), (2, '0.052*"bracket" + 0.035*"corner" + 0.031*"post" + 0.027*"leg" + 0.023*"hole" + 0.023*"nose" + 0.023*"side" + 0.022*"said" + 0.018*"lock" + 0.018*"latch"')]


Document Similarity

In [247]:
docs = []
for doc in token_docs:
    flat_doc = [word for sentence in doc for word in sentence]
    docs.append(flat_doc)
print("Number of documents:",len(docs))

Number of documents: 3


In [250]:
dictionary = gensim.corpora.Dictionary(docs)
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

Number of words in dictionary: 167
0 A
1 adapt
2 adjac
3 align
4 anoth
5 attach
6 back
7 compris
8 direct
9 dispos
10 extend
11 finger
12 glove
13 golf
14 hand
15 index
16 least
17 middl
18 nearli
19 one
20 overlap
21 parallel
22 player
23 portion
24 receiv
25 receptacl
26 restrain
27 said
28 section
29 side
30 strap
31 substanti
32 surfac
33 thumb
34 use
35 wrist
36 In
37 adjust
38 along
39 andd
40 around
41 arrang
42 away
43 bracket
44 combin
45 comprisinga
46 connect
47 corner
48 crib
49 engag
50 extens
51 flush
52 form
53 height
54 hole
55 latch
56 leg
57 legc
58 length
59 lengthwis
60 lie
61 like
62 lock
63 mean
64 nose
65 opposit
66 pivot
67 posit
68 post
69 postb
70 project
71 rel
72 rigid
73 select
74 seri
75 slidabl
76 slot
77 swung
78 toward
79 type
80 across
81 also
82 assembl
83 associ
84 axi
85 bed
86 bedand
87 cross
88 detach
89 effect
90 enabl
91 end
92 extent
93 fingeractuat
94 frame
95 framea
96 frameand
97 gener
98 given
99 groov
100 holessaid
101 includ
102 interconn

In [251]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print(corpus)

[[(0, 3), (1, 2), (2, 3), (3, 1), (4, 1), (5, 2), (6, 4), (7, 5), (8, 3), (9, 1), (10, 4), (11, 17), (12, 3), (13, 4), (14, 2), (15, 6), (16, 2), (17, 4), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 15), (26, 3), (27, 11), (28, 1), (29, 1), (30, 3), (31, 1), (32, 3), (33, 1), (34, 1), (35, 2)], [(2, 1), (10, 2), (19, 1), (27, 2), (29, 5), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 8), (44, 1), (45, 1), (46, 3), (47, 6), (48, 1), (49, 2), (50, 2), (51, 1), (52, 1), (53, 1), (54, 3), (55, 2), (56, 6), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 3), (65, 1), (66, 2), (67, 2), (68, 5), (69, 1), (70, 1), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 2), (77, 2), (78, 2), (79, 1)], [(0, 7), (2, 1), (3, 5), (7, 1), (9, 2), (10, 21), (19, 7), (21, 5), (23, 9), (24, 4), (27, 119), (29, 54), (38, 1), (41, 4), (45, 13), (46, 6), (49, 16), (52, 7), (54, 15), (55, 9), (58, 1), (62, 7), (63, 7), (65, 12), (66, 56), (67, 5),

In [252]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=3, num_nnz=201)
201


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word', 
           stop_words='english', vocabulary=terms)

vect.fit(doc_list)
corpus_tf_idf = vect.transform(doc_list) 

In [16]:
print(corpus_tf_idf)

  (0, 165)	0.23490464527530794
  (0, 161)	0.13873846761367903
  (0, 156)	0.13873846761367903
  (0, 140)	0.2911582530450494
  (0, 128)	0.1055141806670843
  (0, 126)	0.30384198647478483
  (0, 103)	0.1055141806670843
  (0, 102)	0.23490464527530794
  (0, 95)	0.1055141806670843
  (0, 56)	0.408711059807519
  (0, 51)	0.23490464527530794
  (0, 49)	0.3620294173041447
  (0, 48)	0.33107082293693685
  (0, 39)	0.4216064170090753
  (1, 160)	0.1132714136875297
  (1, 145)	0.1917851747230801
  (1, 135)	0.1807866968918071
  (1, 126)	0.17457119184021352
  (1, 121)	0.23771278071945898
  (1, 105)	0.34881269679418087
  (1, 88)	0.2955747212712469
  (1, 72)	0.20556930772201182
  (1, 71)	0.1132714136875297
  (1, 68)	0.08614582973139068
  (1, 66)	0.33368740717993667
  :	:
  (2, 128)	0.04097755453371049
  (2, 127)	0.1285748760559809
  (2, 126)	0.2071464853412102
  (2, 123)	0.25263944988379644
  (2, 118)	0.0538805598130989
  (2, 109)	0.23342160423307193
  (2, 103)	0.1310144444743144
  (2, 99)	0.2688092533275026
 